In [ ]:
from torch_snippets import *
from PIL import Image

import json
with open('/home/pawan/TII_IR-Tracking/src/Daytime/daytime.json', 'r') as f:
   DF_RAW = df = json.load(f)

IMAGE_ROOT = "/home/pawan/TII_IR-Tracking/src/Daytime/RGB/"


In [ ]:
supercategory = {ann['supercategory'] for ann in DF_RAW['categories']}

label2target = {l:t+1 for t,l in enumerate(supercategory)}
label2target['background'] = 0

target2label = {t:l for l,t in label2target.items()}

background_class = label2target['background']
num_classes = len(label2target)



In [ ]:
def preprocess_image(img):
    img = torch.tensor(img).permute(2,0,1)
    return img.to(device).float()

In [ ]:
class OpenDataset(torch.utils.data.Dataset):
    w, h = 224, 224
    def __init__(self, df, image_dir=IMAGE_ROOT):
        self.image_dir = image_dir
        self.files = Glob(self.image_dir+'/*')
        self.df = df
        self.image_infos = [i['file_name'] for i in df["images"]]

        self.image_ids_annotations = defaultdict(list)
        for ann in df['annotations']:
            image_id = ann['image_id'] # Are integers
            self.image_ids_annotations[image_id].append(ann)

        self.catergory_id_to_name = dict()
        for ann in df['categories']:
            self.catergory_id_to_name[ann['id']] = ann['supercategory']


    def __getitem__(self, ix):
        # load images and masks
        image_id = self.image_infos[ix]
        img_path = find(image_id, self.files)
        img = Image.open(img_path).convert("RGB")
        h, w = img.size
        img = np.array(img.resize((self.w, self.h), resample=Image.BILINEAR))/255.

        image_anns = self.image_ids_annotations[ix]
        boxes = list()
        labels = list()

        for image_ann in image_anns:
            bbox = image_ann['bbox']
            
            reformat = [bbox[0] * self.h/h,
                        bbox[1] * self.w/w,
                        (bbox[0]+bbox[2]) * self.h/h,
                        (bbox[1]+bbox[3]) * self.w/w]
            
            
            reformat = list(map(int, reformat))
            boxes.append(reformat) # Format: x, y, h, w, origin in top left corner
            labels.append(self.catergory_id_to_name[image_ann['category_id']])
        
        # torch FRCNN expects ground truths as a dictionary of tensors
        target = {}
        target["boxes"] = torch.Tensor(boxes).float()
        target["labels"] = torch.Tensor([label2target[i] for i in labels]).long()
        img = preprocess_image(img)

        return img, target
    
    def collate_fn(self, batch):
        return tuple(zip(*batch)) 

    def __len__(self):
        return len(self.image_infos)

In [ ]:
from sklearn.model_selection import train_test_split

image_ids = [i['id'] for i in df["images"]]
trn_ids, val_ids = train_test_split(image_ids, test_size=0.1, random_state=99)

image_ids_annotations = defaultdict(list)

for ann in df['annotations']:

    image_id = ann['image_id'] # Are integers
    image_ids_annotations[image_id].append(ann)
    



trn_df = {'images':[],'image_ids_annotations':{}, 'categories':df['categories']}
val_df = {'images':[],'image_ids_annotations':{}, 'categories':df['categories']}


for key in trn_ids:
    if key in image_ids_annotations:
        trn_df['image_ids_annotations'][key] = image_ids_annotations[key]

print(df['images'][5622])

for i in trn_ids:
    try:
        trn_df['images'].append(df['images'][i])
    
    except:
        print(i)

for key in val_ids:
    if key in image_ids_annotations:
        val_df['image_ids_annotations'][key] = image_ids_annotations[key]


len(trn_df), len(val_df)

train_ds = OpenDataset(trn_df)
test_ds = OpenDataset(val_df)

train_loader = DataLoader(train_ds, batch_size=4, collate_fn=train_ds.collate_fn, drop_last=True)
test_loader = DataLoader(test_ds, batch_size=4, collate_fn=test_ds.collate_fn, drop_last=True)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [ ]:
# Defining training and validation functions for a single batch
def train_batch(inputs, model, optimizer):
    model.train()
    input, targets = inputs
    input = list(image.to(device) for image in input)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    optimizer.zero_grad()
    losses = model(input, targets)
    loss = sum(loss for loss in losses.values())
    loss.backward()
    optimizer.step()
    return loss, losses

@torch.no_grad() # this will disable gradient computation in the function below
def validate_batch(inputs, model):
    model.train() # to obtain the losses, model needs to be in train mode only. # #Note that here we are not defining the model's forward method 
#and hence need to work per the way the model class is defined
    input, targets = inputs
    input = list(image.to(device) for image in input)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    optimizer.zero_grad()
    losses = model(input, targets)
    loss = sum(loss for loss in losses.values())
    return loss, losses

In [ ]:
model = get_model().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
n_epochs = 5
log = Report(n_epochs)

In [ ]:
for epoch in range(n_epochs):
    _n = len(train_loader)
    for ix, inputs in enumerate(train_loader):
        loss, losses = train_batch(inputs, model, optimizer)
        loc_loss, regr_loss, loss_objectness, loss_rpn_box_reg = \
            [losses[k] for k in ['loss_classifier','loss_box_reg','loss_objectness','loss_rpn_box_reg']]
        pos = (epoch + (ix+1)/_n)
        log.record(pos, trn_loss=loss.item(), trn_loc_loss=loc_loss.item(), 
                   trn_regr_loss=regr_loss.item(), trn_objectness_loss=loss_objectness.item(),
                   trn_rpn_box_reg_loss=loss_rpn_box_reg.item(), end='\r')

    _n = len(test_loader)
    for ix,inputs in enumerate(test_loader):
        loss, losses = validate_batch(inputs, model)
        loc_loss, regr_loss, loss_objectness, loss_rpn_box_reg = \
          [losses[k] for k in ['loss_classifier','loss_box_reg','loss_objectness','loss_rpn_box_reg']]
        pos = (epoch + (ix+1)/_n)
        log.record(pos, val_loss=loss.item(), val_loc_loss=loc_loss.item(), 
                  val_regr_loss=regr_loss.item(), val_objectness_loss=loss_objectness.item(),
                  val_rpn_box_reg_loss=loss_rpn_box_reg.item(), end='\r')
    if (epoch+1)%(n_epochs//5)==0: log.report_avgs(epoch+1)

In [ ]:
log.plot_epochs(['trn_loss','val_loss'])

In [ ]:
from torchvision.ops import nms
def decode_output(output):
    'convert tensors to numpy arrays'
    bbs = output['boxes'].cpu().detach().numpy().astype(np.uint16)
    labels = np.array([target2label[i] for i in output['labels'].cpu().detach().numpy()])
    confs = output['scores'].cpu().detach().numpy()
    ixs = nms(torch.tensor(bbs.astype(np.float32)), torch.tensor(confs), 0.05)
    bbs, confs, labels = [tensor[ixs] for tensor in [bbs, confs, labels]]

    if len(ixs) == 1:
        bbs, confs, labels = [np.array([tensor]) for tensor in [bbs, confs, labels]]
    return bbs.tolist(), confs.tolist(), labels.tolist()

In [ ]:
model.eval()
for ix, (images, targets) in enumerate(test_loader):
    if ix==3: break
    images = [im for im in images]
    outputs = model(images)
    for ix, output in enumerate(outputs):
        bbs, confs, labels = decode_output(output)
        info = [f'{l}@{c:.2f}' for l,c in zip(labels, confs)]
        show(images[ix].cpu().permute(1,2,0), bbs=bbs, texts=labels, sz=5)